#Boston 311 Tutorial

This notebook will run you through the basic usage of this package to train 3 models on the Boston 311 mydata and use them to predict the outcome of cases from the last 30 days

In [337]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=19321 sha256=279249bcd389db049d3e2ab696c7377ba572aae593761f4ecfb8df870fa407c4
  Stored in directory: /tmp/pip-ephem-wheel-cache-t8qtqgzi/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


##Import the Boston311Model class

In [338]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import sys
import time

In [339]:
from boston311 import Boston311KerasNLP


## Load extra features

In [340]:
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

#format tomorrows date as yyyy-mm-dd
tomorrows_date =  now + pd.DateOffset(days=1)
tomorrows_datestring = tomorrows_date.strftime("%Y-%m-%d")

In [341]:
print('today_datestring: ', today_datestring)
print('tomorrows_datestring: ', tomorrows_datestring)

today_datestring:  2023-10-26
tomorrows_datestring:  2023-10-27


In [342]:
#set path to mydata
#EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_service_id.csv'
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'


In [343]:
import glob
import os

def get_latest_model_files(model_dir="./daily_models/Boston311KerasNLP/"):
    # Get all json and h5 files
    json_files = glob.glob(os.path.join(model_dir, "*.json"))
    h5_files = glob.glob(os.path.join(model_dir, "*.h5"))

    # Sort files by modification time
    json_files.sort(key=os.path.getmtime, reverse=True)
    h5_files.sort(key=os.path.getmtime, reverse=True)

    if json_files and h5_files:
        latest_json = json_files[0]
        latest_h5 = h5_files[0]
        return latest_json, latest_h5
    else:
        return None, None

# Use the function
json_file, model_file = get_latest_model_files()
print(json_file)

if json_file and model_file:
    kerasNLP_model = Boston311KerasNLP()
    kerasNLP_model.load(json_file, model_file)
    kerasNLP_model.predict_date_range['end'] = tomorrows_datestring


./daily_models/Boston311KerasNLP/20231023_042716_Boston311KerasNLP.json


In [344]:
#load data from all_311_data.csv
mydata = pd.read_csv('all_311_cases.csv', low_memory=False)

In [345]:
data = kerasNLP_model.load_data( data=mydata, train_or_predict='predict' )
data = kerasNLP_model.enhance_data( data, 'predict')

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['closed_dt'] = pd.to_datetime(data['closed_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['open_dt'] = pd.to_datetime(data['open_dt'])
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [346]:
cyclical_df = data[['case_enquiry_id', 'season_cos', 'weekday_cos']].copy()

In [347]:
#show first ten records with newest open_dt
data.sort_values(by=['open_dt'], ascending=False).head(10)

,case_enquiry_id,open_dt,sla_target_dt,closed_dt,on_time,case_status,closure_reason,case_title,subject,reason,...,latitude,longitude,geom_4326,source,survival_time,event,ward_number,season_cos,weekday_cos,survival_time_hours
2732470,101005141997,2023-10-25 19:50:00,2023-10-27 04:30:00,NaT,ONTIME,Open,,Missed Trash: District 1A,Public Works Department,Sanitation,...,42.376650,-71.065050,0101000020E610000055473EC929C451C0D2AB3C0D3630...,Constituent Call,NaT,0,2,0.386405,-0.222521,NaN
2732469,101005141994,2023-10-25 19:41:08,2023-11-24 18:41:09,NaT,ONTIME,Open,,Rodent Activity,Inspectional Services,Environmental Services,...,42.352750,-71.081491,0101000020E6100000FA75FA2337C551C09AE5B3E5262D...,Citizens Connect App,NaT,0,5,0.386405,-0.222521,NaN
2732468,101005141988,2023-10-25 19:13:48,NaN,NaT,ONTIME,Open,,Schedule Bulk Item Pickup,Public Works Department,Sanitation,...,42.299400,-71.084531,0101000020E6100000DBCF30F368C551C0286D8DB95226...,Self Service,NaT,0,14,0.386405,-0.222521,NaN
2732467,101005141987,2023-10-25 19:10:17,2023-10-27 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.309300,-71.090731,0101000020E6100000A226D787CEC551C0F53DD4209727...,Citizens Connect App,NaT,0,12,0.386405,-0.222521,NaN
2732466,101005141986,2023-10-25 19:05:57,2023-11-09 03:30:00,NaT,ONTIME,Open,,Sign Repair,Transportation - Traffic Division,Signs & Signals,...,42.356574,-71.067615,0101000020E61000001EE141CF53C451C0AE8C5E3BA42D...,Citizens Connect App,NaT,0,5,0.386405,-0.222521,NaN
2732465,101005141985,2023-10-25 18:54:23,2023-10-27 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.318906,-71.055549,0101000020E6100000D90A691C8EC351C0E45B55EBD128...,Citizens Connect App,NaT,0,13,0.386405,-0.222521,NaN
2732464,101005141983,2023-10-25 18:48:13,2023-10-27 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.315908,-71.063420,0101000020E6100000DD6FE2100FC451C05C6F70A86F28...,Citizens Connect App,NaT,0,13,0.386405,-0.222521,NaN
2732463,101005141982,2023-10-25 18:44:00,2023-10-30 04:30:00,NaT,ONTIME,Open,,Improper Storage of Trash (Barrels),Public Works Department,Code Enforcement,...,42.337609,-71.027590,0101000020E6100000AEA4A00AC4C151C06D7B97C9362B...,Constituent Call,NaT,0,6,0.386405,-0.222521,NaN
2732462,101005141978,2023-10-25 18:38:11,2023-10-27 04:30:00,NaT,ONTIME,Open,,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,42.374510,-71.066060,0101000020E6100000B4A984553AC451C0BFDBA0EDEF2F...,Citizens Connect App,NaT,0,2,0.386405,-0.222521,NaN
2732461,101005141977,2023-10-25 18:34:00,2023-10-27 04:30:00,NaT,ONTIME,Open,,Request for Pothole Repair,Public Works Department,Highway Maintenance,...,42.387164,-71.072401,0101000020E610000093797B39A2C451C0F94CC8938E31...,Constituent Call,NaT,0,2,0.386405,-0.222521,NaN


In [348]:
clean_data = kerasNLP_model.clean_data_for_prediction( data )

In [349]:
clean_data.head()

,case_enquiry_id,queue_ANML02_LostFound,queue_ANML_General,queue_BHA_General,queue_BPD_Administrative,queue_BPD_Districts,queue_BPS_Administrative,queue_BPS_Transportation Administration,queue_BTDT_AVRS Interface Queue,queue_BTDT_Abandoned Bicycle,...,department_GEN_,department_GRNi,department_INFO,department_ISD,department_No Q,department_ONS_,department_PARK,department_PROP,department_PWDx,department_Temp
0,101005077769,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,101005077771,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,101005077772,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,101005077773,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,101005077776,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [350]:
mydata = clean_data

In [351]:
mydata['case_enquiry_id']

0        101005077769
1        101005077771
2        101005077772
3        101005077773
4        101005077776
             ...     
27085    101005141986
27086    101005141987
27087    101005141988
27088    101005141994
27089    101005141997
Name: case_enquiry_id, Length: 27090, dtype: object

In [352]:
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

pickle_file = 'dataframe.pkl'

X = None

if os.path.exists(pickle_file):
    #check if the file date is earlier than EXTRA_mydata_FILE date
    pickle_file_date = os.path.getmtime(pickle_file)
    EXTRA_mydata_FILE_date = os.path.getmtime(EXTRA_mydata_FILE)
    if pickle_file_date < EXTRA_mydata_FILE_date:
        os.remove(pickle_file)

if os.path.exists(pickle_file):

    X = pickle.load(open(pickle_file, "rb"))
else:
    X = pd.read_csv(EXTRA_mydata_FILE)

    #rename service_request_id to case_enquiry_id
    X.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)
    #remove all rows where case_enquiry_id is non-numeric
    #X = X[X['case_enquiry_id'].str.isnumeric()]
    #convert case_enquiry_id to int64
    #X['case_enquiry_id'] = X['case_enquiry_id'].astype('int64')

    # Convert stringified arrays back to NumPy arrays
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        X[col] = X[col].apply(literal_eval).apply(np.array)

    pickle.dump(X, open(pickle_file, "wb"))



In [353]:
#concatenate the two dataframes and reindex
df = X

In [354]:
df.shape

(291036, 7)

In [355]:


# Assuming df is your DataFrame and it has columns 'desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding'
desc_cls_embedding_flattened = np.stack(df['desc_cls_embedding'].to_numpy())
desc_pooled_embedding_flattened = np.stack(df['desc_pooled_embedding'].to_numpy())
#do the same for the rest
name_cls_embedding_flattened = np.stack(df['name_cls_embedding'].to_numpy())
name_pooled_embedding_flattened = np.stack(df['name_pooled_embedding'].to_numpy())
code_cls_embedding_flattened = np.stack(df['code_cls_embedding'].to_numpy())
code_pooled_embedding_flattened = np.stack(df['code_pooled_embedding'].to_numpy())


# Remove the old columns
df.drop(['desc_cls_embedding', 'desc_pooled_embedding'], axis=1, inplace=True)
#do the same for the rest
df.drop(['name_cls_embedding', 'name_pooled_embedding'], axis=1, inplace=True)
df.drop(['code_cls_embedding', 'code_pooled_embedding'], axis=1, inplace=True)


# Add the new flattened columns
df_desc_cls = pd.DataFrame(desc_cls_embedding_flattened, columns=[f'desc_cls_{i}' for i in range(desc_cls_embedding_flattened.shape[1])])

df_desc_pooled = pd.DataFrame(desc_pooled_embedding_flattened, columns=[f'desc_pooled_{i}' for i in range(desc_pooled_embedding_flattened.shape[1])])
#do the same for the rest
df_name_cls = pd.DataFrame(name_cls_embedding_flattened, columns=[f'name_cls_{i}' for i in range(name_cls_embedding_flattened.shape[1])])
df_name_pooled = pd.DataFrame(name_pooled_embedding_flattened, columns=[f'name_pooled_{i}' for i in range(name_pooled_embedding_flattened.shape[1])])
df_code_cls = pd.DataFrame(code_cls_embedding_flattened, columns=[f'code_cls_{i}' for i in range(code_cls_embedding_flattened.shape[1])])
df_code_pooled = pd.DataFrame(code_pooled_embedding_flattened, columns=[f'code_pooled_{i}' for i in range(code_pooled_embedding_flattened.shape[1])])



df = pd.concat([df, df_desc_cls, df_desc_pooled, df_name_cls, df_name_pooled, df_code_cls, df_code_pooled], axis=1)

In [356]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
is_numeric = df['case_enquiry_id'].str.isnumeric()

In [357]:
df = df[is_numeric]

In [358]:
df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')

In [359]:
df.shape

(291036, 769)

In [360]:
mydata.shape

(27090, 267)

In [361]:
#join them so we are left only with records that have mydata in both files
new_mydata = mydata.merge(df, on='case_enquiry_id', how='inner')


In [362]:
new_mydata = new_mydata.merge(cyclical_df, on='case_enquiry_id', how='inner')

In [363]:
new_mydata.shape

(17716, 1037)

In [364]:

df = new_mydata

In [365]:
#cast all columns that are type bool to float
for col in df.columns:
    if df[col].dtype == 'bool':
        df[col] = df[col].astype('float64')
    if df[col].dtype == 'int64':
        df[col] = df[col].astype('float64') 

In [366]:
#list the number of rows in X and y
print(df.dtypes)


case_enquiry_id              object
queue_ANML02_LostFound      float64
queue_ANML_General          float64
queue_BHA_General           float64
queue_BPD_Administrative    float64
                             ...   
code_pooled_125             float64
code_pooled_126             float64
code_pooled_127             float64
season_cos                  float64
weekday_cos                 float64
Length: 1037, dtype: object


In [367]:
#free all unused dataframes
#df_to_delete = [cls_embedding_flattened, pooled_embedding_flattened, df_cls, df_pooled, X, new_mydata, is_numeric, mydata]

#for data_frame in df_to_delete:
#    if data_frame is not None:
#        del data_frame

In [368]:
case_enquiry_id = df['case_enquiry_id']
X_predict = df.drop(['case_enquiry_id'], axis=1)


In [369]:

#parse CLS embedding column as array
predictions = kerasNLP_model.model.predict(X_predict)

122/554 [=====>........................] - ETA: 1s

554/554 [==============================] - 2s 3ms/step


In [370]:
# Define a function to flatten an array into a string.
def array_to_string(arr):
    return ' '.join(map(str, arr))

# Apply the function along axis 1 (rows).
string_predictions = np.apply_along_axis(array_to_string, axis=1, arr=predictions)

# Now string_predictions is a 1D NumPy array where each element is a string
# that contains all the elements from the corresponding row in the original 2D array.
print(string_predictions)

['0.99999994 1.4331269e-24 4.316801e-34 4.717023e-30 1.4092472e-30 1.7594578e-31 3.16442e-30 1.4702165e-31 1.9724375e-33 1.3482826e-38 1.3247428e-32 8.392188e-28 2.0035094e-30 1.03236797e-25 1.1477308e-30 1.3861533e-26 3.038414e-30 1.7402166e-35 2.7055124e-25 2.0018074e-36 1.9393258e-33 8.104516e-32 2.3639786e-33 2.336285e-34 3.3072745e-31 1.6130883e-35 5.599027e-27 9.400286e-33 7.713507e-29 2.0373193e-31 5.038065e-25 9.4748805e-33 4.5197993e-32 4.86418e-29 1.9261895e-31 8.514725e-28 4.894775e-24 2.3788126e-27 1.1784661e-25 1.8188756e-27 6.132769e-26 3.109363e-31 9.069626e-34 6.8113787e-37 2.7898407e-32 6.4764294e-27 4.352503e-33 3.2693452e-29 6.0599464e-29 6.624901e-36 9.627237e-35 4.2983828e-33 3.5509543e-36 1.3257573e-27 3.101288e-36 3.6120976e-33 2.0133075e-31 1.8224977e-38 4.637208e-27 0.0 1.7698724e-30'
 '0.98676556 3.6806416e-08 1.4923833e-06 2.0235322e-13 6.009686e-05 1.212138e-13 4.6078985e-06 2.5999067e-10 1.4554942e-07 6.042216e-06 0.00012423274 1.674283e-07 4.045736e-05 3.2

In [371]:
#combine case_enquiry_id and predictions into a dataframe
predictions_df = pd.DataFrame({'case_enquiry_id':case_enquiry_id, 'prediction':string_predictions})

In [372]:
bin_labels = [
    "0-12 hours",      # Less than half a day
    "12-24 hours",     # Half to one day
    "1-3 days",        # One to three days
    "4-7 days",        # Four to seven days
    "1-2 weeks",       # One to two weeks
    "2-4 weeks",       # Two to four weeks
    "1-2 months",      # One to two months
    "2-4 months",      # Two to four months
    "4+ months"        # More than four months
]

#predictions_df['prediction'] = predictions_df['prediction'].apply(lambda x: bin_labels[x])

In [373]:
len(predictions_df)

17716

In [374]:
print(predictions)

[[9.9999994e-01 1.4331269e-24 4.3168010e-34 ... 4.6372079e-27
  0.0000000e+00 1.7698724e-30]
 [9.8676556e-01 3.6806416e-08 1.4923833e-06 ... 1.0271908e-21
  4.1296451e-16 1.2805920e-02]
 [9.9999994e-01 5.0088864e-30 3.0234434e-36 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [8.4646910e-01 4.3630791e-03 1.4914076e-01 ... 1.7629930e-14
  1.2071371e-18 2.8430532e-06]
 [8.4245557e-01 1.5754285e-01 1.7677712e-07 ... 7.0474269e-13
  3.9115719e-15 9.6292709e-08]
 [9.9597037e-01 3.6897638e-03 1.2755593e-05 ... 2.9858591e-11
  1.8216231e-11 3.0051699e-04]]


In [375]:
kerasNLP_model.model_type

'Boston311KerasNLP'

In [376]:
import pandas as pd

#get model_name from json_file name and ml_model_date from json_file name first 8 characters which are YYYYMMDD and change it to YYYY-MM-DD
model_name = json_file.split('/')[-1].split('.')[0]
ml_model_date = model_name[:4] + '-' + model_name[4:6] + '-' + model_name[6:8]

#define an empt pandas dataframe ml_model_df
ml_model_df = pd.DataFrame(columns=['ml_model_name', 'ml_model_type', 'ml_model_date'])

ml_model_df = pd.concat([ml_model_df, pd.DataFrame([{'ml_model_name': model_name, 
                                    'ml_model_type': kerasNLP_model.model_type,
                                    'ml_model_date': ml_model_date}])], ignore_index=True)

print(ml_model_df)

                       ml_model_name      ml_model_type ml_model_date
0  20231023_042716_Boston311KerasNLP  Boston311KerasNLP    2023-10-23


In [377]:
model_cases = data.drop(['geom_4326','survival_time_hours', 'survival_time', 'event','season_cos', 'weekday_cos'], axis=1).copy()

In [378]:
all_model_cases = model_cases 
all_model_predictions = predictions_df

In [379]:
all_model_predictions['ml_model_name'] = model_name

In [380]:
all_model_predictions['prediction_date'] = today_datestring

In [381]:
# %%
#get current datetime in Boston timezone as string
from datetime import datetime
from pytz import timezone
import pytz
boston = timezone('US/Eastern')
now = datetime.now(boston)
today_datestring = now.strftime("%Y-%m-%d")
#get time in Boston timezone as string for a filename
now = datetime.now(boston)
time_string = now.strftime("%H-%M-%S")
#define datetime string
my_datetime = today_datestring + '_' + time_string 

In [382]:


# %%
all_model_cases.to_csv(my_datetime+'_311_cases.csv', index=False)


# %%

all_model_predictions.to_csv(my_datetime+'_311_predictions.csv', index=False)

# %%

ml_model_df.to_csv(my_datetime+'_311_ml_models.csv', index=False)

# %%
#create datetime _manifest.txt file with one filename per line
with open(my_datetime+'_manifest.txt', 'w') as f:
    f.write(my_datetime+'_311_cases.csv\n')
    f.write(my_datetime+'_311_predictions.csv\n')
    f.write(my_datetime+'_311_ml_models.csv\n')

# %%
#create an export folder
EXPORT_FOLDER = '~/Documents/BODC-DEI-site/database/seeders'
#copy the csv files to the export folder
!cp {my_datetime}_311_cases.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_predictions.csv {EXPORT_FOLDER}
!cp {my_datetime}_311_ml_models.csv {EXPORT_FOLDER}
!cp {my_datetime}_manifest.txt {EXPORT_FOLDER}



# %% [markdown]
# ** Copy the files to the production server **

# %%
# Define constants for servers
PROD_USER = 'u353344964'
PROD_HOSTNAME = '195.179.236.61'
PORT_NUMBER = 65002
PROD_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel'
STAGE_BASE_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel'
PROD_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/laravel/database/seeders'
STAGE_EXPORT_FOLDER = '/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders'

# %%







In [383]:
import os


def scp_to_server(filename, user=PROD_USER, hostname=PROD_HOSTNAME, port=PORT_NUMBER, export_folder=PROD_EXPORT_FOLDER):
    """Copy a file to the server using scp."""
    command = f"scp -P {port} {filename} {user}@{hostname}:{export_folder}"
    print(f"Executing: {command}")
    os.system(command)

# Use the function to scp files
files_to_copy = [
    f"{my_datetime}_311_cases.csv",
    f"{my_datetime}_311_predictions.csv",
    f"{my_datetime}_311_ml_models.csv",
    f"{my_datetime}_manifest.txt"
]

# Control where to copy
copy_to_prod = True
copy_to_stage = True

for file in files_to_copy:
    if copy_to_prod:
        scp_to_server(file, export_folder=PROD_EXPORT_FOLDER)
    if copy_to_stage:
        scp_to_server(file, export_folder=STAGE_EXPORT_FOLDER)


# %%
PORT_NUMBER

Executing: scp -P 65002 2023-10-26_15-35-56_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders


Executing: scp -P 65002 2023-10-26_15-35-56_311_cases.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_311_predictions.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_311_ml_models.csv u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/stagelaravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_manifest.txt u353344964@195.179.236.61:/home/u353344964/domains/bodc-dei.org/laravel/database/seeders
Executing: scp -P 65002 2023-10-26_15-35-56_manifest.txt u353344964@195.179.236.61:/

65002

In [384]:

if copy_to_prod:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {PROD_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'

if copy_to_stage:
    ! ssh -p {PORT_NUMBER} {PROD_USER}@{PROD_HOSTNAME} 'cd {STAGE_BASE_FOLDER}; php artisan db:seed --class=ThreeOneOneSeeder'


   INFO  Seeding database.  


Manifest files:
/home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-26_15-35-56_manifest.txt

Total records to process: 44807

Processing /home/u353344964/domains/bodc-dei.org/laravel/database/seeders/2023-10-26_15-35-56_311_cases.csv
100 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 26990.
Total records remaining: 44707.
Time for last 100 records: 0.03 seconds.
Estimated time remaining for this file: 7 seconds.
Estimated time for all files: 12 seconds.
200 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 26890.
Total records remaining: 44607.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 second.
Estimated time for all files: 2 seconds.
300 App\Models\ThreeOneOneCase records processed.
Records remaining in this file: 26790.
Total records remaining: 44507.
Time for last 100 records: 0.01 seconds.
Estimated time remaining for this file: 1 se